https://www.kaggle.com/vbmokin/very-significant-safe-memory-lightgbm?select=weather_train.csv

In [1]:
# This Python 3 environment comes with many helpful analytics libraries installed
# It is defined by the kaggle/python Docker image: https://github.com/kaggle/docker-python
# For example, here's several helpful packages to load

import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)

# Input data files are available in the read-only "../input/" directory
# For example, running this (by clicking run or pressing Shift+Enter) will list all files under the input directory

import os
for dirname, _, filenames in os.walk('/kaggle/input'):
    for filename in filenames:
        print(os.path.join(dirname, filename))

# You can write up to 5GB to the current directory (/kaggle/working/) that gets preserved as output when you create a version using "Save & Run All" 
# You can also write temporary files to /kaggle/temp/, but they won't be saved outside of the current session

/kaggle/input/ashrae-energy-prediction/weather_test.csv
/kaggle/input/ashrae-energy-prediction/building_metadata.csv
/kaggle/input/ashrae-energy-prediction/train.csv
/kaggle/input/ashrae-energy-prediction/test.csv
/kaggle/input/ashrae-energy-prediction/sample_submission.csv
/kaggle/input/ashrae-energy-prediction/weather_train.csv


In [2]:
submission = pd.read_csv("../input/ashrae-energy-prediction/sample_submission.csv")
test = pd.read_csv("../input/ashrae-energy-prediction/test.csv")
train = pd.read_csv("../input/ashrae-energy-prediction/train.csv")
weather_train = pd.read_csv("../input/ashrae-energy-prediction/weather_train.csv")
weather_test = pd.read_csv("../input/ashrae-energy-prediction/weather_test.csv")
metadata = pd.read_csv("../input/ashrae-energy-prediction/building_metadata.csv")

In [3]:
train = train.merge(metadata, left_on = "building_id", right_on = "building_id", how = "left")
train = train.merge(weather_train, left_on = ["site_id", "timestamp"], right_on = ["site_id", "timestamp"])
del weather_train

In [4]:
train.shape

(20125605, 16)

In [5]:
train

,building_id,meter,timestamp,meter_reading,site_id,primary_use,square_feet,year_built,floor_count,air_temperature,cloud_coverage,dew_temperature,precip_depth_1_hr,sea_level_pressure,wind_direction,wind_speed
0,0,0,2016-01-01 00:00:00,0.000,0,Education,7432,2008.0,NaN,25.0,6.0,20.0,NaN,1019.7,0.0,0.0
1,1,0,2016-01-01 00:00:00,0.000,0,Education,2720,2004.0,NaN,25.0,6.0,20.0,NaN,1019.7,0.0,0.0
2,2,0,2016-01-01 00:00:00,0.000,0,Education,5376,1991.0,NaN,25.0,6.0,20.0,NaN,1019.7,0.0,0.0
3,3,0,2016-01-01 00:00:00,0.000,0,Education,23685,2002.0,NaN,25.0,6.0,20.0,NaN,1019.7,0.0,0.0
4,4,0,2016-01-01 00:00:00,0.000,0,Education,116607,1975.0,NaN,25.0,6.0,20.0,NaN,1019.7,0.0,0.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
20125600,1444,0,2016-12-31 23:00:00,8.750,15,Entertainment/public assembly,19619,1914.0,NaN,1.7,NaN,-5.6,-1.0,1008.5,180.0,8.8
20125601,1445,0,2016-12-31 23:00:00,4.825,15,Education,4298,NaN,NaN,1.7,NaN,-5.6,-1.0,1008.5,180.0,8.8
20125602,1446,0,2016-12-31 23:00:00,0.000,15,Entertainment/public assembly,11265,1997.0,NaN,1.7,NaN,-5.6,-1.0,1008.5,180.0,8.8
20125603,1447,0,2016-12-31 23:00:00,159.575,15,Lodging/residential,29775,2001.0,NaN,1.7,NaN,-5.6,-1.0,1008.5,180.0,8.8


In [6]:
train["timestamp"] = pd.to_datetime(train["timestamp"])

train["weekday"] = train["timestamp"].dt.weekday
train["hour"] = train["timestamp"].dt.hour

train["weekday"] = train['weekday'].astype(np.uint8)
train["hour"] = train['hour'].astype(np.uint8)

train['year_built'] = train['year_built']-1900
train['square_feet'] = (10*np.log(train['square_feet'])).astype(np.uint8)

In [7]:
def average_imputation(df, column_name):
    imputation = df.groupby(['timestamp'])[column_name].mean()
    
    df.loc[df[column_name].isnull(), column_name] = df[df[column_name].isnull()][[column_name]].apply(lambda x: imputation[df['timestamp'][x.index]].values)
    del imputation
    return df

In [8]:
train = average_imputation(train, 'wind_speed')
train = average_imputation(train, 'wind_direction')

In [9]:
train

,building_id,meter,timestamp,meter_reading,site_id,primary_use,square_feet,year_built,floor_count,air_temperature,cloud_coverage,dew_temperature,precip_depth_1_hr,sea_level_pressure,wind_direction,wind_speed,weekday,hour
0,0,0,2016-01-01 00:00:00,0.000,0,Education,89,108.0,NaN,25.0,6.0,20.0,NaN,1019.7,0.0,0.0,4,0
1,1,0,2016-01-01 00:00:00,0.000,0,Education,79,104.0,NaN,25.0,6.0,20.0,NaN,1019.7,0.0,0.0,4,0
2,2,0,2016-01-01 00:00:00,0.000,0,Education,85,91.0,NaN,25.0,6.0,20.0,NaN,1019.7,0.0,0.0,4,0
3,3,0,2016-01-01 00:00:00,0.000,0,Education,100,102.0,NaN,25.0,6.0,20.0,NaN,1019.7,0.0,0.0,4,0
4,4,0,2016-01-01 00:00:00,0.000,0,Education,116,75.0,NaN,25.0,6.0,20.0,NaN,1019.7,0.0,0.0,4,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
20125600,1444,0,2016-12-31 23:00:00,8.750,15,Entertainment/public assembly,98,14.0,NaN,1.7,NaN,-5.6,-1.0,1008.5,180.0,8.8,5,23
20125601,1445,0,2016-12-31 23:00:00,4.825,15,Education,83,NaN,NaN,1.7,NaN,-5.6,-1.0,1008.5,180.0,8.8,5,23
20125602,1446,0,2016-12-31 23:00:00,0.000,15,Entertainment/public assembly,93,97.0,NaN,1.7,NaN,-5.6,-1.0,1008.5,180.0,8.8,5,23
20125603,1447,0,2016-12-31 23:00:00,159.575,15,Lodging/residential,103,101.0,NaN,1.7,NaN,-5.6,-1.0,1008.5,180.0,8.8,5,23


In [10]:
beaufort = [(0, 0, 0.3), (1, 0.3, 1.6), (2, 1.6, 3.4), (3, 3.4, 5.5), (4, 5.5, 8), (5, 8, 10.8), (6, 10.8, 13.9), 
          (7, 13.9, 17.2), (8, 17.2, 20.8), (9, 20.8, 24.5), (10, 24.5, 28.5), (11, 28.5, 33), (12, 33, 200)]

for item in beaufort:
    train.loc[(train['wind_speed']>=item[1]) & (train['wind_speed']<item[2]), 'beaufort_scale'] = item[0]

In [11]:
train

,building_id,meter,timestamp,meter_reading,site_id,primary_use,square_feet,year_built,floor_count,air_temperature,cloud_coverage,dew_temperature,precip_depth_1_hr,sea_level_pressure,wind_direction,wind_speed,weekday,hour,beaufort_scale
0,0,0,2016-01-01 00:00:00,0.000,0,Education,89,108.0,NaN,25.0,6.0,20.0,NaN,1019.7,0.0,0.0,4,0,0.0
1,1,0,2016-01-01 00:00:00,0.000,0,Education,79,104.0,NaN,25.0,6.0,20.0,NaN,1019.7,0.0,0.0,4,0,0.0
2,2,0,2016-01-01 00:00:00,0.000,0,Education,85,91.0,NaN,25.0,6.0,20.0,NaN,1019.7,0.0,0.0,4,0,0.0
3,3,0,2016-01-01 00:00:00,0.000,0,Education,100,102.0,NaN,25.0,6.0,20.0,NaN,1019.7,0.0,0.0,4,0,0.0
4,4,0,2016-01-01 00:00:00,0.000,0,Education,116,75.0,NaN,25.0,6.0,20.0,NaN,1019.7,0.0,0.0,4,0,0.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
20125600,1444,0,2016-12-31 23:00:00,8.750,15,Entertainment/public assembly,98,14.0,NaN,1.7,NaN,-5.6,-1.0,1008.5,180.0,8.8,5,23,5.0
20125601,1445,0,2016-12-31 23:00:00,4.825,15,Education,83,NaN,NaN,1.7,NaN,-5.6,-1.0,1008.5,180.0,8.8,5,23,5.0
20125602,1446,0,2016-12-31 23:00:00,0.000,15,Entertainment/public assembly,93,97.0,NaN,1.7,NaN,-5.6,-1.0,1008.5,180.0,8.8,5,23,5.0
20125603,1447,0,2016-12-31 23:00:00,159.575,15,Lodging/residential,103,101.0,NaN,1.7,NaN,-5.6,-1.0,1008.5,180.0,8.8,5,23,5.0


In [12]:
def degToCompass(num):
    val=int((num/22.5)+.5)
    arr=[i for i in range(0,16)]
    return arr[(val % 16)]
train['wind_direction'] = train['wind_direction'].apply(degToCompass)

train['beaufort_scale'] = train['beaufort_scale'].astype(np.uint8)
train["wind_direction"] = train['wind_direction'].astype(np.uint8)
train["meter"] = train['meter'].astype(np.uint8)
train["site_id"] = train['site_id'].astype(np.uint8)
train["building_id"] = train['building_id'].astype(np.uint16)

train['air_temperature'] = np.int8(round(2*train['air_temperature'],0)) # store values with precision 0.5
train['cloud_coverage'] = np.uint8(round(5*train['cloud_coverage'],0)) # store values with precision 0.2
train['dew_temperature'] = np.int8(round(5*train['dew_temperature'],0)) # store values with precision 0.2
train['precip_depth_1_hr'] = np.uint8(np.clip(round(train['precip_depth_1_hr'],0),0,255)) # transform [-1,343] to [0,255]

In [13]:
train

,building_id,meter,timestamp,meter_reading,site_id,primary_use,square_feet,year_built,floor_count,air_temperature,cloud_coverage,dew_temperature,precip_depth_1_hr,sea_level_pressure,wind_direction,wind_speed,weekday,hour,beaufort_scale
0,0,0,2016-01-01 00:00:00,0.000,0,Education,89,108.0,NaN,50,30,100,0,1019.7,0,0.0,4,0,0
1,1,0,2016-01-01 00:00:00,0.000,0,Education,79,104.0,NaN,50,30,100,0,1019.7,0,0.0,4,0,0
2,2,0,2016-01-01 00:00:00,0.000,0,Education,85,91.0,NaN,50,30,100,0,1019.7,0,0.0,4,0,0
3,3,0,2016-01-01 00:00:00,0.000,0,Education,100,102.0,NaN,50,30,100,0,1019.7,0,0.0,4,0,0
4,4,0,2016-01-01 00:00:00,0.000,0,Education,116,75.0,NaN,50,30,100,0,1019.7,0,0.0,4,0,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
20125600,1444,0,2016-12-31 23:00:00,8.750,15,Entertainment/public assembly,98,14.0,NaN,3,0,-28,0,1008.5,8,8.8,5,23,5
20125601,1445,0,2016-12-31 23:00:00,4.825,15,Education,83,NaN,NaN,3,0,-28,0,1008.5,8,8.8,5,23,5
20125602,1446,0,2016-12-31 23:00:00,0.000,15,Entertainment/public assembly,93,97.0,NaN,3,0,-28,0,1008.5,8,8.8,5,23,5
20125603,1447,0,2016-12-31 23:00:00,159.575,15,Lodging/residential,103,101.0,NaN,3,0,-28,0,1008.5,8,8.8,5,23,5


In [14]:
train = train.fillna(0)
train['year_built'] = train['year_built'].astype(np.uint8)
train['floor_count'] = train['floor_count'].astype(np.uint8)

In [15]:
from sklearn.preprocessing import LabelEncoder

le = LabelEncoder()
train["primary_use"] = le.fit_transform(train["primary_use"]).astype(np.uint8)

categoricals = ["site_id", "building_id", "primary_use", "hour", "weekday", "meter",  "wind_direction"]
drop_cols = ["sea_level_pressure", "wind_speed"]
numericals = ["square_feet", "year_built", "air_temperature", "cloud_coverage","dew_temperature", 'precip_depth_1_hr', 'floor_count', 'beaufort_scale']

feat_cols = categoricals + numericals

In [16]:
target = np.log1p(train["meter_reading"]).astype(np.float16)
train = train.drop(drop_cols, axis = 1)
del train["meter_reading"]

In [17]:
train.describe()

,building_id,meter,site_id,primary_use,square_feet,year_built,floor_count,air_temperature,cloud_coverage,dew_temperature,precip_depth_1_hr,wind_direction,weekday,hour,beaufort_scale
count,2.012560e+07,2.012560e+07,2.012560e+07,2.012560e+07,2.012560e+07,2.012560e+07,2.012560e+07,2.012560e+07,2.012560e+07,2.012560e+07,2.012560e+07,2.012560e+07,2.012560e+07,2.012560e+07,2.012560e+07
mean,7.972116e+02,6.617305e-01,7.967773e+00,3.159801e+00,1.098921e+02,2.720936e+01,7.252062e-01,3.196674e+01,5.378028e+00,3.898024e+01,7.030669e-01,6.996953e+00,3.007445e+00,1.151151e+01,2.313889e+00
std,4.263729e+02,9.310748e-01,5.092223e+00,3.451352e+00,1.182405e+01,3.850382e+01,2.295898e+00,2.189968e+01,1.019208e+01,5.059926e+01,6.626986e+00,4.856641e+00,1.996799e+00,6.923813e+00,1.300147e+00
min,0.000000e+00,0.000000e+00,0.000000e+00,0.000000e+00,5.600000e+01,0.000000e+00,0.000000e+00,-5.800000e+01,0.000000e+00,-1.280000e+02,0.000000e+00,0.000000e+00,0.000000e+00,0.000000e+00,0.000000e+00
25%,3.910000e+02,0.000000e+00,3.000000e+00,0.000000e+00,1.030000e+02,0.000000e+00,0.000000e+00,1.700000e+01,0.000000e+00,0.000000e+00,0.000000e+00,3.000000e+00,1.000000e+00,6.000000e+00,2.000000e+00
50%,8.940000e+02,0.000000e+00,9.000000e+00,1.000000e+00,1.110000e+02,0.000000e+00,0.000000e+00,3.300000e+01,0.000000e+00,4.400000e+01,0.000000e+00,7.000000e+00,3.000000e+00,1.200000e+01,2.000000e+00
75%,1.176000e+03,1.000000e+00,1.300000e+01,6.000000e+00,1.180000e+02,6.400000e+01,0.000000e+00,4.800000e+01,1.000000e+01,8.000000e+01,0.000000e+00,1.100000e+01,5.000000e+00,1.800000e+01,3.000000e+00
max,1.448000e+03,3.000000e+00,1.500000e+01,1.500000e+01,1.360000e+02,1.170000e+02,2.600000e+01,9.400000e+01,4.500000e+01,1.270000e+02,2.550000e+02,1.500000e+01,6.000000e+00,2.300000e+01,8.000000e+00


In [18]:
train.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 20125605 entries, 0 to 20125604
Data columns (total 16 columns):
 #   Column             Dtype         
---  ------             -----         
 0   building_id        uint16        
 1   meter              uint8         
 2   timestamp          datetime64[ns]
 3   site_id            uint8         
 4   primary_use        uint8         
 5   square_feet        uint8         
 6   year_built         uint8         
 7   floor_count        uint8         
 8   air_temperature    int8          
 9   cloud_coverage     uint8         
 10  dew_temperature    int8          
 11  precip_depth_1_hr  uint8         
 12  wind_direction     uint8         
 13  weekday            uint8         
 14  hour               uint8         
 15  beaufort_scale     uint8         
dtypes: datetime64[ns](1), int8(2), uint16(1), uint8(12)
memory usage: 1.2 GB


In [19]:
train.columns

Index(['building_id', 'meter', 'timestamp', 'site_id', 'primary_use',
       'square_feet', 'year_built', 'floor_count', 'air_temperature',
       'cloud_coverage', 'dew_temperature', 'precip_depth_1_hr',
       'wind_direction', 'weekday', 'hour', 'beaufort_scale'],
      dtype='object')

In [20]:
from sklearn.metrics import mean_squared_error
import lightgbm as lgb
from sklearn.model_selection import KFold, StratifiedKFold
from tqdm import tqdm


params = {
            'boosting_type': 'gbdt',
            'objective': 'regression',
            'metric': {'rmse'},
            'subsample': 0.25,
            'subsample_freq': 1,
            'learning_rate': 0.4,
            'num_leaves': 17,
            'feature_fraction': 0.9,
            'lambda_l1': 1,  
            'lambda_l2': 1}

folds = 4
seed = 666

kf = StratifiedKFold(n_splits=folds, shuffle=True, random_state=seed)

models = []
for train_index, val_index in kf.split(train, train['building_id']):
    train_X = train[feat_cols].iloc[train_index]
    val_X = train[feat_cols].iloc[val_index]
    train_y = target.iloc[train_index]
    val_y = target.iloc[val_index]
    lgb_train = lgb.Dataset(train_X, train_y, categorical_feature=categoricals)
    lgb_eval = lgb.Dataset(val_X, val_y, categorical_feature=categoricals)
    gbm = lgb.train(params,
                lgb_train,
                num_boost_round=500,
                valid_sets=(lgb_train, lgb_eval),
                early_stopping_rounds=100,
                verbose_eval = 100)
    models.append(gbm)

/opt/conda/lib/python3.7/site-packages/lightgbm/basic.py:1291: UserWarning: Using categorical_feature in Dataset.
  warnings.warn('Using categorical_feature in Dataset.')


Training until validation scores don't improve for 100 rounds
[100]	training's rmse: 1.06361	valid_1's rmse: 1.06401
[200]	training's rmse: 1.02888	valid_1's rmse: 1.02994
[300]	training's rmse: 1.00989	valid_1's rmse: 1.01192
[400]	training's rmse: 0.997578	valid_1's rmse: 1.00032
[500]	training's rmse: 0.987485	valid_1's rmse: 0.991027
Did not meet early stopping. Best iteration is:
[500]	training's rmse: 0.987485	valid_1's rmse: 0.991027
Training until validation scores don't improve for 100 rounds
[100]	training's rmse: 1.0445	valid_1's rmse: 1.04563
[200]	training's rmse: 1.02076	valid_1's rmse: 1.02278
[300]	training's rmse: 1.00751	valid_1's rmse: 1.0105
[400]	training's rmse: 0.996792	valid_1's rmse: 1.00081
[500]	training's rmse: 0.987654	valid_1's rmse: 0.992418
Did not meet early stopping. Best iteration is:
[500]	training's rmse: 0.987654	valid_1's rmse: 0.992418
Training until validation scores don't improve for 100 rounds
[100]	training's rmse: 1.06689	valid_1's rmse: 1.0

In [21]:
import gc
del train, train_X, val_X, lgb_train, lgb_eval, train_y, val_y, target
gc.collect()

244

s

In [22]:
test = test.merge(metadata, left_on = "building_id", right_on = "building_id", how = "left")
del metadata
gc.collect()

20

In [23]:
le = LabelEncoder()
test["primary_use"] = le.fit_transform(test["primary_use"]).astype(np.uint8)
test = test.merge(weather_test, left_on = ["site_id", "timestamp"], right_on = ["site_id", "timestamp"], how = "left")
del weather_test
gc.collect()

20

In [24]:
test["timestamp"] = pd.to_datetime(test["timestamp"])
test["hour"] = test["timestamp"].dt.hour
test["weekday"] = test["timestamp"].dt.weekday
test["weekday"] = test['weekday'].astype(np.uint8)
test["hour"] = test['hour'].astype(np.uint8)
test['year_built'] = test['year_built']-1900
test['square_feet'] = (10*np.log(test['square_feet'])).astype(np.uint8)
test["building_id"] = test['building_id'].astype(np.uint16)

test['air_temperature'] = np.int8(round(2*test['air_temperature'],0)) # store values with precision 0.5
test['cloud_coverage'] = np.uint8(round(10*test['cloud_coverage'],0)) # store values with precision 0.1
test['dew_temperature'] = np.int8(round(5*test['dew_temperature'],0)) # store values with precision 0.2
test['precip_depth_1_hr'] = np.uint8(np.clip(round(test['precip_depth_1_hr'],0),0,255)) # transform [-1,343] to [0,255]

test = average_imputation(test, 'wind_speed')
test = average_imputation(test, 'wind_direction')

for item in beaufort:
    test.loc[(test['wind_speed']>=item[1]) & (test['wind_speed']<item[2]), 'beaufort_scale'] = item[0]
test['wind_direction'] = test['wind_direction'].apply(degToCompass)

test['wind_direction'] = test['wind_direction'].apply(degToCompass)
test['beaufort_scale'] = test['beaufort_scale'].astype(np.uint8)
test["wind_direction"] = test['wind_direction'].astype(np.uint8)
test["meter"] = test['meter'].astype(np.uint8)
test["site_id"] = test['site_id'].astype(np.uint8)

test = test[feat_cols]

In [25]:
test = test.fillna(0)
test['year_built'] = test['year_built'].astype(np.uint8)
test['floor_count'] = test['floor_count'].astype(np.uint8)

In [26]:
i=0
res=[]
step_size = 50000
for j in tqdm(range(int(np.ceil(test.shape[0]/50000)))):
    res.append(np.expm1(sum([model.predict(test.iloc[i:i+step_size]) for model in models])/folds))
    i+=step_size

100%|██████████| 834/834 [28:53<00:00,  2.08s/it]


In [27]:
res = np.concatenate(res)
submission['meter_reading'] = res
submission.loc[submission['meter_reading']<0, 'meter_reading'] = 0
submission.to_csv('submission_new.csv', index=False)
submission

,row_id,meter_reading
0,0,17.640354
1,1,7.304295
2,2,1.439131
3,3,15.304415
4,4,48.220885
...,...,...
41697595,41697595,9.295866
41697596,41697596,5.689063
41697597,41697597,3.982140
41697598,41697598,185.515179
